# Assignment V2

## Imports

In [ ]:
#checks for environment
import sys
print(sys.executable)



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
from typing import List, Dict, Any, Tuple
import os
from dotenv import load_dotenv
import json
import panel as pn
from openai import OpenAI

load_dotenv() 

client = OpenAI()  
print(os.getenv("OPENAI_API_KEY") is not None)

In [ ]:
#pour import
PROJECT_ROOT = Path.cwd()
while not (PROJECT_ROOT / "data").exists() and PROJECT_ROOT != PROJECT_ROOT.parent:
    PROJECT_ROOT = PROJECT_ROOT.parent

print("PROJECT_ROOT =", PROJECT_ROOT)

# Dossiers d'entrée
DATA_DIR = PROJECT_ROOT / "data"
TXT_DIR  = DATA_DIR / "fiches_produit_txt"
CSV_DIR  = DATA_DIR / "fiches_produit_csv"

print("TXT_DIR =", TXT_DIR)
print("CSV_DIR =", CSV_DIR)

In [ ]:
# chemins d'exports 
PRODUCTS_JSON_PATH = PROJECT_ROOT / "output_data" / "products_json" / "products.json"
PRODUCTS_CSV_PATH  = PROJECT_ROOT / "output_data" / "products_csv"  / "products.csv"

print("JSON export:", PRODUCTS_JSON_PATH)
print("CSV export :", PRODUCTS_CSV_PATH)


## Data loading


Récuperer la donnée des fichiers txt, puis csv et la display

In [ ]:
def load_csv_products(csv_dir: Path) -> pd.DataFrame:
    """Charge tous les fichiers CSV et les concatène dans un seul DataFrame."""
    csv_paths = sorted(csv_dir.glob("*.csv"))
    print("Fichiers CSV trouvés :", [p.name for p in csv_paths])

    frames = []
    for path in csv_paths:
        df = pd.read_csv(path)
        df["raw_file"] = path.name  
        frames.append(df)

    if not frames:
        return pd.DataFrame()

    return pd.concat(frames, ignore_index=True)


In [ ]:
df_csv_raw = load_csv_products(CSV_DIR)
print("CSV products shape :", df_csv_raw.shape)


In [ ]:
df_csv_raw.head()

In [ ]:
def load_txt_products(txt_dir: Path) -> pd.DataFrame:
    """Charge tous les fichiers TXT dans un DataFrame (texte brut)."""
    txt_paths = sorted(txt_dir.glob("*.txt"))
    records = []

    for path in txt_paths:
        text = path.read_text(encoding="utf-8")
        records.append(
            {
                "raw_file": path.name,
                "raw_text": text,
            }
        )

    return pd.DataFrame(records)

In [ ]:
df_txt_raw = load_txt_products(TXT_DIR)

In [ ]:
print("TXT products shape :", df_txt_raw.shape)

In [ ]:
df_txt_raw.head()

In [ ]:
print(df_txt_raw.loc[0, "raw_text"])

## CSV File extraction

In [ ]:

# Schéma cible pour un produit
PRODUCT_FIELDS = [
    "product_name",
    "price",
    "category",
    "colors",
    "description_short",
    "features",
    "in_stock",
    "raw_file",
]

In [ ]:
df_csv_raw["description"][0]

In [ ]:
df_csv_raw

In [ ]:
#Parsing wantes format

def parse_price(value) -> float:
    """Convertit le prix en float (gère '129', '129.0', '129,99', etc.)."""
    if pd.isna(value):
        return None
    if isinstance(value, (int, float)):
        return float(value)
    s = str(value).strip()
    s = s.replace("€", "").replace("eur", "").replace("euros", "")
    s = s.replace(",", ".")
    s = s.strip()
    return float(s) if s else None


def parse_colors(value) -> List[str]:
    """Convertit une chaîne de couleurs en liste normalisée."""
    if pd.isna(value):
        return []
    s = str(value).lower()
    parts = re.split(r"[,/;]|et", s)
    return [p.strip() for p in parts if p.strip()]


def parse_features(value) -> List[str]:
    """Convertit une chaîne de features en liste."""
    if pd.isna(value):
        return []
    s = str(value)
    # souvent séparé par ';' ou par '•' ou '-'
    parts = re.split(r"[;•\n\-]+", s)
    return [p.strip() for p in parts if p.strip()]


def parse_in_stock(value) -> bool:
    """Transforme une info de stock texte en booléen."""
    s = str(value).lower()
    if "rupture" in s:
        return False
    if "épuisé" in s:
        return False
    # si on voit "stock", "dispo", etc. on considère True
    if "stock" in s or "dispo" in s or "oui" in s:
        return True
    # fallback : True par défaut si on n'a pas d'info claire
    return True

In [ ]:
#creating a dataframe
def normalize_csv_products(df_csv: pd.DataFrame) -> pd.DataFrame:
    """Transforme le df brut CSV en tableau standardisé."""
    products: List[Dict[str, Any]] = []

    for _, row in df_csv.iterrows():
        product = {
            "product_name": str(row.get("product_name", "")).strip(),
            "price": parse_price(row.get("price")),
            "category": str(row.get("category", "")).strip().lower(),
            "colors": parse_colors(row.get("colors")),
            # pour l'instant, on prend la description telle quelle,
            # plus tard on utilisera une API pour faire un vrai résumé
            "description_short": str(row.get("description", "")).strip(),
            "features": parse_features(row.get("features")),
            "in_stock": parse_in_stock(row.get("stock", "")),
            "raw_file": row.get("raw_file"),
        }
        products.append(product)

    return pd.DataFrame(products, columns=PRODUCT_FIELDS)


In [ ]:

df_products_csv = normalize_csv_products(df_csv_raw)

print("df_products_csv shape :", df_products_csv.shape)
df_products_csv.head()


In [ ]:
df_products_csv["description_short"][0]

## TXT files extraction using api

In [ ]:
df_txt_raw


In [ ]:
df_txt_raw["raw_text"][0]

In [ ]:
def extract_product_from_text_with_openai(raw_text: str, raw_file: str) -> Dict[str, Any]:
    system_msg = (
        "Tu es un assistant qui extrait des informations structurées de fiches produit. "
        "Tu dois répondre SEULEMENT avec un JSON valide, sans texte autour."
    )

    user_msg = f"""
Voici le contenu d'une fiche produit :

\"\"\"{raw_text}\"\"\"

Extrait les informations suivantes :

- product_name
- price (float, en euros)
- category
- colors (liste de couleurs)
- description_short (résumé en 1–2 phrases)
- features (liste de caractéristiques)
- in_stock (booléen)
- raw_file (nom du fichier), qui doit être: "{raw_file}"
"""

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user",  "content": user_msg},
        ],
        temperature=0,
        response_format={"type": "json_object"},  # force le JSON
    )

    content = response.choices[0].message.content

    try:
        data = json.loads(content)
    except json.JSONDecodeError as e:
        print("⚠️ Erreur de parsing JSON pour le fichier:", raw_file)
        print("Contenu renvoyé par le modèle :")
        print(content)
        raise e

    # S'assurer que toutes les clés attendues existent
    for field in PRODUCT_FIELDS:
        data.setdefault(field, None)

    # Normalisations de base
    if data["colors"] is None:
        data["colors"] = []
    if data["features"] is None:
        data["features"] = []

    # in_stock → bool
    if isinstance(data["in_stock"], str):
        s = data["in_stock"].lower()
        data["in_stock"] = not ("rupture" in s or "épuisé" in s)

    # price → float si string
    if isinstance(data["price"], str):
        s = (
            data["price"]
            .replace("€", "")
            .replace("euros", "")
            .replace("euro", "")
            .replace(",", ".")
            .strip()
        )
        data["price"] = float(s) if s else None

    return data

In [ ]:

products_txt: List[Dict[str, Any]] = []

for _, row in df_txt_raw.iterrows():
    product = extract_product_from_text_with_openai(
        raw_text=row["raw_text"],
        raw_file=row["raw_file"],
    )
    products_txt.append(product)

df_products_txt = pd.DataFrame(products_txt, columns=PRODUCT_FIELDS)
df_products_txt.head()


In [ ]:
row0 = df_txt_raw.iloc[0]
test_product = extract_product_from_text_with_openai(
    raw_text=row0["raw_text"],
    raw_file=row0["raw_file"],
)
test_product


## Normalisation and validation

Now that we have the good columns, and that our features are well parsed, let's Join txt and csv data frame

In [ ]:
df_all_products = pd.concat(
    [df_products_csv, df_products_txt],
    ignore_index=True
)

df_all_products


In [ ]:
df_all_products.info()
df_all_products.head()


In [ ]:
ALLOWED_CATEGORIES = {
    "furniture",
    "electronics",
    "electronics / audio",
    "electronics / lighting",
    "accessory",
    "wearable",
    "office",
    "clothing",
}

In [ ]:
df_all_products["category"].unique()


In [ ]:

def validate_product(product: Dict[str, Any]) -> Dict[str, Any]:
    """
    Vérifie qu'un produit respecte les règles de validation.
    Retourne un dict avec:
      - validation_status: "ok" ou "error"
      - validation_errors: liste de messages d'erreurs
    """
    errors: List[str] = []

    # 1. price > 0
    price = product.get("price")
    if not isinstance(price, (int, float)) or price <= 0:
        errors.append("price must be a positive number")

    # 2. category dans une liste autorisée
    category_raw = product.get("category")
    category = (category_raw or "").strip().lower()
    if not category:
        errors.append("category is missing")
    elif category not in ALLOWED_CATEGORIES:
        errors.append(f"category '{category}' is not allowed")

    # 3. product_name non vide
    name = (product.get("product_name") or "").strip()
    if not name:
        errors.append("product_name is empty")

    # 4. description_short ≤ 280 caractères
    desc = product.get("description_short") or ""
    if len(desc) == 0:
        errors.append("description_short is empty")
    elif len(desc) > 280:
        errors.append("description_short exceeds 280 characters")

    # 5. in_stock booléen
    in_stock = product.get("in_stock")
    if not isinstance(in_stock, bool):
        errors.append("in_stock must be a boolean")

    # 6. features liste non vide si présente
    features = product.get("features")
    if features is not None:
        if not isinstance(features, list):
            errors.append("features must be a list if present")
        elif len(features) == 0:
            errors.append("features list is empty")

    status = "ok" if not errors else "error"

    return {
        "validation_status": status,
        "validation_errors": errors,
    }

In [ ]:
def apply_validation(row: pd.Series) -> pd.Series:
    result = validate_product(row.to_dict())
    return pd.Series(result)

df_validated = df_all_products.copy()

df_validated[["validation_status", "validation_errors"]] = df_validated.apply(
    apply_validation,
    axis=1,
)

df_validated

In [ ]:
df_all_products["description_short"]

In [ ]:
print(df_all_products["description_short"][0])


In [ ]:
def afficher_descriptions(df):
    for i in range(len(df)):
        print(f"Ligne {i} :")
        print(df['description_short'][i])
        print("-" * 50)


In [ ]:
afficher_descriptions(df_all_products)

## API use for ChatBot : summarize, translate

In [ ]:
catalog_df = df_validated.copy()  # adapte si tu utilises un autre df

def build_catalog_for_llm(df) -> str:
    products: List[Dict[str, Any]] = []
    for idx, row in df.iterrows():
        products.append(
            {
                "id": int(idx),
                "name": row["product_name"],
                "category": row["category"],
                "price_eur": row["price"],
                "colors": row["colors"],
                "description": row["description_short"],
                "features": row["features"],
                "in_stock": row["in_stock"],
            }
        )
    return json.dumps(products, ensure_ascii=False, indent=2)

CATALOG_TEXT = build_catalog_for_llm(catalog_df)


In [ ]:
CATALOG_TEXT

In [ ]:
def get_completion_from_messages(messages, model="gpt-4.1-mini", temperature=0.2):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content


In [ ]:
context: List[Dict[str, str]] = [
    {
        "role": "system",
        "content": f"""
Tu es un assistant pour un petit catalogue de produits.

Par défaut, tu réponds en FRANÇAIS.
MAIS :
- si la question de l'utilisateur est entièrement dans une autre langue,
  tu réponds dans cette langue ;
- si l'utilisateur demande explicitement une autre langue (par exemple :
  "résume ce produit en anglais", "réponds en espagnol"), tu suis sa demande.
On te fournit ci-dessous la liste des produits au format JSON, avec :
- id
- name
- category
- price_eur
- colors
- description
- features
- in_stock

Tu utilises UNIQUEMENT ces informations pour répondre.

L'utilisateur peut poser des questions comme :
- Donne les informations résumées du produit 2
- Quels produits font partie de la catégorie wearable ?
- Donne-moi les détails sur la chaise de bureau
- Quels produits sont adaptés au télétravail ?

Dans ta réponse, pour chaque produit pertinent, donne :
- le nom du produit
- la catégorie (traduite en français)
- le prix
- la disponibilité (en stock ou non)
- un résumé très court.

Catalogue JSON :
{CATALOG_TEXT}
"""
    }
]




In [ ]:
# --- Panel : UI du chatbot catalogue ----------------------------------------

pn.extension()

# colonne qui va contenir toutes les lignes de conversation
conversation = pn.Column()

In [ ]:
# zone de texte + bouton
inp = pn.widgets.TextInput(
    value="",
    placeholder="Pose ta question sur les produits…",
    width=600,
)
button_conversation = pn.widgets.Button(
    name="Chat!",
    button_type="primary",
)


def on_chat_click(event):
    """Callback déclenché quand on clique sur le bouton Chat!"""
    global context

    question = inp.value.strip()
    if not question:
        return

    # on vide la zone de texte
    inp.value = ""

    # on affiche la question dans la conversation
    conversation.append(
        pn.Row("User:", pn.pane.Markdown(question, width=600))
    )

    # on appelle le modèle, avec gestion d'erreur
    try:
        # on ajoute la question dans le contexte
        context.append({"role": "user", "content": question})
        answer = get_completion_from_messages(context)
        # on ajoute la réponse dans le contexte pour la suite de la conversation
        context.append({"role": "assistant", "content": answer})
    except Exception as e:
        answer = f"⚠️ Erreur lors de l'appel au modèle : `{e}`"

    # on affiche la réponse de l'assistant, avec un style lisible sur fond sombre
    conversation.append(
        pn.Row(
            "Assistant:",
            pn.pane.Markdown(
                answer,
                width=600,
                styles={
                    "background-color": "#222222",
                    "color": "white",
                    "padding": "10px",
                    "border-radius": "4px",
                },
            ),
        )
    )


# on relie le bouton au callback (plus de pn.bind ici)
button_conversation.on_click(on_chat_click)

dashboard = pn.Column(
    "## Assistant catalogue produits",
    pn.pane.Markdown(
        "Pose tes questions en français ou en anglais, par exemple :  \n"
        "- *Quels produits y a-t-il dans la catégorie wearable ?*  \n"
        "- *Peux-tu me résumer les informations concernant la chaise de bureau ?*  \n"
        "- *Quels produits sont adaptés au télétravail ?*  \n"
    ),
    pn.Row(inp, button_conversation),
    pn.Spacer(height=10),
    conversation,
)

dashboard

## Data export

In [ ]:
CSV_COLUMNS = [
    "product_name",
    "price",
    "category",
    "in_stock",
    "validation_status",
]

In [ ]:
def make_products_json(df: pd.DataFrame) -> str:
    """
    Retourne une chaîne JSON indentée contenant toutes les colonnes du DataFrame.
    """
    records = df.to_dict(orient="records")
    return json.dumps(records, ensure_ascii=False, indent=2)


# ================== 2) CSV (DataFrame filtré) ==================

def make_products_csv_frame(df: pd.DataFrame) -> pd.DataFrame:
    """
    Retourne un DataFrame ne contenant que les colonnes nécessaires pour le CSV.
    """
    missing = [col for col in CSV_COLUMNS if col not in df.columns]
    if missing:
        raise ValueError(f"Colonnes manquantes dans le DataFrame : {missing}")

    return df[CSV_COLUMNS].copy()


# ================== 3) Export global ==================

def export_products(df_validated: pd.DataFrame) -> Tuple[Path, Path]:
    """
    - Génère le JSON complet et le CSV filtré à partir de df_validated.
    - Crée les dossiers si nécessaire.
    - Écrit :
        * products.json dans output_data/products_json/
        * products.csv dans output_data/products_csv/
    - Affiche les chemins générés.
    - Retourne (json_path, csv_path).
    """
    # Préparation des contenus
    json_str = make_products_json(df_validated)
    csv_frame = make_products_csv_frame(df_validated)

    # Création des dossiers si besoin
    PRODUCTS_JSON_PATH.parent.mkdir(parents=True, exist_ok=True)
    PRODUCTS_CSV_PATH.parent.mkdir(parents=True, exist_ok=True)

    # Écritures des fichiers
    PRODUCTS_JSON_PATH.write_text(json_str, encoding="utf-8")
    csv_frame.to_csv(PRODUCTS_CSV_PATH, index=False, encoding="utf-8")

    # Logs minimalistes
    print(f"JSON exporté vers : {PRODUCTS_JSON_PATH}")
    print(f"CSV exporté vers  : {PRODUCTS_CSV_PATH}")

    return PRODUCTS_JSON_PATH, PRODUCTS_CSV_PATH

json_path, csv_path = export_products(df_validated)